# 추천 시스템: 콘텐츠 기반 추천 시스템

In [ ]:
import warnings
warnings.filterwarnings(action='ignore')

In [ ]:
import pandas as pd
pd.set_option('max_colwidth', 100)

In [ ]:
df = pd.read_csv('data/movie/tmdb_5000_movies.csv')
df.head()

In [ ]:
moview_df = df[['id', 'title','genres', 'vote_average', 'vote_count', 'popularity', 'keywords', 'overview']]
moview_df.head()

In [ ]:
moview_df.info()

In [ ]:
moview_df = moview_df.drop(['id'], axis=1)

In [ ]:
from ast import literal_eval

result = literal_eval(moview_df.iloc[0]['genres'])
result

### 데이터 전처리

##### 장르

In [ ]:
moview_df['genres'] = moview_df['genres'].apply(literal_eval)
moview_df['genres']

In [ ]:
import re

name_result = []
for genre_temp in moview_df['genres']:
    temp_name = []
    for temp in genre_temp:
        name = temp['name']
        temp_name.append(re.sub('[\s]', '', name))
    name_result.append(temp_name)
name_result

moview_df['genres'] = name_result

In [ ]:
data_result = []
for temp in moview_df['genres']:
    data = ' '.join(temp)
    data_result.append(data)

moview_df['genres_literal'] = data_result

##### 키워드

In [ ]:
moview_df['keywords'] = moview_df['keywords'].apply(literal_eval)
moview_df['keywords']

In [ ]:
name_result = []
for keyword_temp in moview_df['keywords']:
    temp_name = []
    for temp in keyword_temp:
        name = temp['name']
        temp_name.append(name)
    name_result.append(temp_name)
name_result

moview_df['keywords'] = name_result

### 카운트 벡터라이징

In [ ]:
moview_df['genres_literal']

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

count_vect = CountVectorizer(min_df = 1, ngram_range = (1, 1)) 
# ngram_range 단어를 1개 에서 2개의 조합을 만들고
# min_df 전체 데이터에서 한번 이상 등장한 조합만 사용

genre_vect = count_vect.fit_transform(moview_df['genres_literal'])

In [ ]:
count_vect.vocabulary_

In [ ]:
genre_vect.shape

In [ ]:
genre_vect_array = genre_vect.toarray()
genre_vect_array

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

genre_sim = cosine_similarity(genre_vect_array, genre_vect_array)
genre_sim[0].argsort()[::-1]

In [ ]:
moview_df['title'].iloc[0]

In [ ]:
moview_df['title'].iloc[46]

In [ ]:
moview_df['title'].iloc[3569]

In [ ]:
def find_sim_movie(df, sorted_index, title_name, top_n = 10):
    title_movie = df[df['title'] == title_name]
    title_index = title_movie.index.values[0]

    similar_index = sorted_index[title_index, : (top_n)]
    similar_index = similar_index.reshape(-1)

    return df.iloc[similar_index]

genre_sim_sorted = genre_sim.argsort()[:, ::-1]

movie_result = find_sim_movie(df, genre_sim_sorted, 'Superman', 10)
print(movie_result['title'])

print(df.iloc[3494]['genres'])
print(df.iloc[813]['genres'])